from config import set_utils_path

set_utils_path()

Só por enquanto, eu vou manter desta maneira, para poder continuar a codar no que eu estava trabalhando:

In [1]:
from directory_area_verde_load import (
    definir_diretorio,
    get_data_diretorio
)

project_path = definir_diretorio()
data_path = get_data_diretorio(project_path)

In [2]:
import os

from utils import (
    get_diretorio_data,
    #get_root_projeto,
    check_crs,
    save_parquet_excel
)
from utils.downloads import (
    download_malha_geosampa
)

# Quadra Viária (praças)

Deu problema ao baixar o arquivo, o que deveria ser salvo como uma pasta compactada, foi baixado como um outro tipo de arquivo. Preciso corrigir isso na função depois. 

 f'''{os.path.join(
                    'vsizip',
                    file_apth
                )}'''

# Importar dados 

In [4]:
gdf_quadra= download_malha_geosampa(
    'quadra_viaria_editada',
    data_path,
    #paginate=True
)

OK, sem a paginação deu certo, vamos tentar com a paginação agora.

Com a paginação está dando errado. Vou confirmar com o Henrique depois qula o motivo disso, já que ele entende melhor o código da paginação melhor que eu.

In [5]:
gdf_quadra.shape

(30000, 8)

In [6]:
gdf_quadra.sample(2)

,cd_identif,cd_identi0,tx_tipo_qu,tx_escala,tx_ano_ref,qt_area_me,sg_fonte_o,geometry
22570,21583,27295,Quadra,1:1.000,2004,4683,SMUL/GEOINFO,"POLYGON ((353360.98 7399654.712, 353362.896 73..."
3739,3471,5181,Quadra,1:1.000,2004,12119,SMUL/GEOINFO,"POLYGON ((325443.117 7378961.715, 325444.344 7..."


## Padronizaçã de colunas

In [7]:
drop_quadra_cols={
    'tx_escala',
    'tx_ano_ref'
}

gdf_quadra.drop(
    columns=drop_quadra_cols,
    axis=1,
    inplace=True
)

In [8]:
rename_quadra_cols = {
    'cd_identif' : 'cd_quadra'
}

gdf_quadra.rename(
    rename_quadra_cols,
    axis=1,
    inplace=True
)

## Conferir CRS

In [9]:
gdf_quadra = check_crs(gdf_quadra)

# Filtrar por praças

In [10]:
gdf_quadra.columns

Index(['cd_quadra', 'cd_identi0', 'tx_tipo_qu', 'qt_area_me', 'sg_fonte_o',
       'geometry'],
      dtype='object')

In [11]:
gdf_quadra['tx_tipo_qu'].unique()

array(['Quadra', 'Praca_Canteiro', 'CET', 'Borda'], dtype=object)

Eu fui orientada a pegar apenas o Praca_Canteiro, mas caso seja interessante, é possível pegar outros tipos também.

In [12]:
praca = gdf_quadra['tx_tipo_qu'].unique()[1]

Por algum motivo, quando eu digitava 'Praca_Canteiro', o loc não reconhecia, por isso passei como uma variável, para pegar o nome certinho.

In [13]:
gdf_quadra = gdf_quadra.loc[gdf_quadra['tx_tipo_qu']==praca]

In [14]:
gdf_quadra.shape

(10926, 6)

Acredito que após a paginação, o número de praças será maior.

In [15]:
gdf_quadra.sample(2)

,cd_quadra,cd_identi0,tx_tipo_qu,qt_area_me,sg_fonte_o,geometry
10563,10348,13845,Praca_Canteiro,5591,SMUL/GEOINFO,"POLYGON ((335500.641 7404639.419, 335504.002 7..."
6489,6333,8865,Praca_Canteiro,195,SMUL/GEOINFO,"POLYGON ((333813.892 7386343.404, 333807.445 7..."


# Salvar GDF

In [16]:
save_parquet_excel(
    gdf_quadra, 
    fname= 'quadra_viaria_editada',
    data_path = data_path,
    data_subpath = 'assets'
)